# HTLM Clones detection

The problem:
This project is about clustering html documents that are similar from the perspective of a user opening them <br>
We have 4 folders containing a different number of documents ad we want to extract from each of them the groups of clones. <br>

First question that should be asked is: "What does it mean that 2 websites are similar?"<br>

There are multiple anwers to this question but I'll focus on 2 of them:<br>
1)  2 websites are simmilar if they look the same.<br>
    The first thing that a user notices when landing on a webpage is its style as in color scheme, images, layout of the elements etc<br>
2)  2 websites are similar if the information that they transmit is similar (the textual content is similar)<br>

Analysing these 2 criteria is enough to decide if one site is the clone of another.

# Scraping for information

But before we get to make the final decision there are some steps to make. The first of them is to collect the raw information from the websites we want to classify.

## Scraping for text data

First we'll define a fuction that given a parameter *path* for a specific document will extract all the text found in certain html elements such as p, li, div, h1 ect <br>
For this we'll use the get_text function of the BeautifulSoup library. <br>
Also, all the links, new lines and carriage returs will be removed from the text because they are irrelevant for the project's purpose.

In [6]:
import re
from bs4 import BeautifulSoup



def extract_text(path):
    with open(path, "r", encoding="utf") as file:
        html = file.read()

    soup = BeautifulSoup(html, "html.parser")
    text = soup.get_text()
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = text.replace('\n', ' ').replace('\r', '')

    return text.strip()


## Scraping for visual information

Using the Selenium library we can open each page in a specific window size, take a screen shot of the whole page and then save each of the images in a folder to be processed later. 

In [5]:
import os
import glob
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

SCREENSHOT_DIR = "screenshots4"
os.makedirs(SCREENSHOT_DIR, exist_ok=True)
TIER1 = "clones/tier1"
TIER2 = "clones/tier2"
TIER3 = "clones/tier3"
TIER4 = "clones/tier4"

tier1_files = glob.glob(os.path.join(TIER1, "*.html"))
tier2_files = glob.glob(os.path.join(TIER2, "*.html"))
tier3_files = glob.glob(os.path.join(TIER3, "*.html"))
tier4_files = glob.glob(os.path.join(TIER4, "*.html"))


def take_screenshot(url, filename):
    options = Options()
    options.add_argument('--headless')

    driver = webdriver.Chrome(options=options)

    try:
        driver.get(url)
        path = os.path.join(SCREENSHOT_DIR, filename)
        driver.set_window_size(1200, 900)

        driver.save_screenshot(path)
        print(f"screenshot taken: {path}")
        return path
    except Exception as e:
        print(f"error: {e}")
    finally:
        driver.quit()

def screenshots_for_tiers(directory):
    for file in directory:
        url = f"file:///{os.path.abspath(file)}"
        domain = os.path.basename(file)
        filename = domain.replace(".", "_").replace("_html", ".png")
        take_screenshot(url, filename)

# Feature Exctraction 
Now we have have to text and the screenshots, but this data is unusable for training a model, thus we need to process the data we aquired so far in order to get an array of numerical value that is representative for the dataset (a feature vector)<br>

- For textual information there are numerous methods in NLP in feature extraction.<br>
For this project we'll use TF-IDF Vectorization. <br>
It outputs the importance of a word in a collection of documents. <br>
how it does that? <br> It calculates The frequency of a term appearing in a document TF: <br>
$$
TF = \frac{Nr \ of \ times \ a \ word \ apears \ in \ document}{Nr \ of \ words \ in \ a \ document}
$$

And the Inverse Document Frequency which is the total number of documents devided by the number of documents containing said term: <br>

$$
IDF = \log {\frac{Nr}{Nr \ of \ documents \ containing \ said \ word}}
$$

Then, the TF-IDF score will just be the product of TF and IDF <br>

There are some other alternatives, such as BoW(is simpler, creates a vector of word frequencies, but the representation si sparse), Hashing vectorizer(best for reducing the dimensionality of the feature matrix, it's best for large datasets, which is not the case.), or word-embeddings(computationally expensive)

In [ ]:
import os
import cv2
import pandas as pd
import glob
import string
import numpy as np
from skimage.feature import local_binary_pattern
from scrape_text import extract_text
from sklearn.feature_extraction.text import TfidfVectorizer


def preprocess_text(text):
    return text.lower().translate(str.maketrans("", "", string.punctuation))

def extract_tfidf_features(data):
    vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
    tfidf_matrix = vectorizer.fit_transform(data.values())
    return {key: tfidf_matrix[i].toarray()[0] for i, key in enumerate(data.keys())}

doc = {
    "doc1" : "this is an example text",
    "doc2" : "Another example of text",
    "doc3" : "Example 3 of no meaning sentance"
    }

print(extract_tfidf_features(doc))

## Feature Extraction for Screenshots

For Images we'll use the Local Binary Pattern. <br>

It assigns for each pixel a number in binary representation of 8 bits as follows: It looks at the neighbours of every pixel clockwise starting from the top left. If the neighbour is higher in intensity it appends a 1 to the number, otherwise a 0

In [ ]:
import matplotlib.pyplot as plt
from skimage import feature, io, color
def lbp_histogram(image, num_points=24, radius=3, bins=32):
    lbp = local_binary_pattern(image, num_points, radius, method="uniform")
    hist, _ = np.histogram(lbp.ravel(), bins=bins, range=(0, bins), density=True)
    return hist

image = io.imread('https://pixelixe.com/docs/image-processing/grayscale-image-api.png')
gray_image = color.rgb2gray(image)
lbp_image = feature.local_binary_pattern(gray_image, 8, 1, method="uniform")

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(gray_image, cmap='gray')
axes[0].set_title('Original Grayscale Image')
axes[0].axis('off')

axes[1].imshow(lbp_image, cmap='gray')
axes[1].set_title('LBP Image')
axes[1].axis('off')

plt.show()


Now we can create the combined feature matrix

In [ ]:
import glob
import os
TIER1 = "clones/tier1"
TIER2 = "clones/tier2"
TIER3 = "clones/tier3"
TIER4 = "clones/tier4"

tier1_files = glob.glob(os.path.join(TIER1, "*.html"))
tier2_files = glob.glob(os.path.join(TIER2, "*.html"))
tier3_files = glob.glob(os.path.join(TIER3, "*.html"))
tier4_files = glob.glob(os.path.join(TIER4, "*.html"))

def extract_features(file):

    if file == 1:
        SCREENSHOT_DIR = "screenshots1"
        tier = tier1_files
        path = "clones/tier1\\"
    elif file == 2:
        SCREENSHOT_DIR = "screenshots2"
        tier = tier2_files
        path = "clones/tier2\\"
    elif file == 3:
        SCREENSHOT_DIR = "screenshots3"
        tier = tier3_files
        path = "clones/tier3\\"
    elif file == 4:
        SCREENSHOT_DIR = "screenshots4"
        tier = tier4_files
        path = "clones/tier4\\"
        
    text = {file: extract_text(file) for file in tier}
    tfidf_features = extract_tfidf_features(text)

    screenshots = glob.glob(os.path.join(SCREENSHOT_DIR, "*.png"))
    texture_features = {}

    for screenshot in screenshots:
        image = cv2.imread(screenshot, cv2.IMREAD_GRAYSCALE)
        histo = lbp_histogram(image)
        texture_features[path + os.path.basename(screenshot).replace("_", ".").replace(".png", ".html")] = histo

    common_keys = set(tfidf_features.keys()) & set(texture_features.keys())
    combined_features = {key: np.concatenate((tfidf_features[key], texture_features[key])) for key in common_keys}

    df = pd.DataFrame.from_dict(combined_features, orient="index")
    shape_text = len(next(iter(tfidf_features.values())))
    shape_img = len(next(iter(texture_features.values())))

    df.to_csv("combined_features.csv", index=True, header=False)

    print("✅ Features saved to combined_features4.csv")
    return shape_text, shape_img, combined_features

# Creating the Model

Now that we have the feature matrix we can finally train a model to cluster the websites <br>
One of the simplest and most effective clustering algorithm is K-means. It works by randomly placing a predifined number of centroids K in the feature plain, then assigning the to that centroids the points closest to them, then moving the centroids to the geometric center of their points. <br>
The process is repeated until the centroids don't move anymore.

Firstly, load the feature matrix

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from kneed import KneeLocator

print("choose tier for clustering: 1, 2, 3 or 4")

tier = 1 # Modifica pentru a alege alt folder
print("Extracting textual and image features...")
shape_text, shape_img, combined_features = extract_features(tier)
feature_matrix = np.array(list(combined_features.values()))

## Scaling
One thing to consider is scaling the features. <br>
Let's consider the next scenario: There are 2 websites identical in text, but completly diferent in aspect. Should they be grouped together or not? <br>
Since it's not good to judge a book by its cover :))) (the information is more important than style) I think they should be grouped. To ensure this the text features will have a higher weight than the rest.<br>
Also, we'll aply the standard scaler transformation before this on the entire matrix, to center and normalize the data on the origin

In [ ]:
text_collums = np.arange(0, shape_text)
scaler = StandardScaler()
scaled_features = scaler.fit_transform(feature_matrix)
scaled_features[:, text_collums] = scaled_features[:, text_collums] * 3
valid_websites = [website for website in enumerate(combined_features.keys())]

## Principal Component Analysis
One thing that can improve the computation speed drastically is aplying the PCA transform<br>
This is used for dimension reduction, as it retains only the features that describe 95% of the variance of the dataset


In [ ]:
pca = PCA(n_components=0.95)
pca_features = pca.fit_transform(scaled_features)

## What is the best k?

One problem that K-means has is that you have to choose a predifined number of centroids before training it. But we don't know how many clusters there are before hand. <br>
So what can we do?<br>
There are technics for finding the best k. One of them is the *Elbow Method*. It works by looking at the graph of the inertia with respect to the number of clusters k, and choosing the k where the graph bends(has a high second derivative).<br>
However, it doesn't work for all datasets. Some have an evolution of inertia almost linear, or an elbow is hard to find<br>
For those, we can chose another method, such as the *Sillhouete Score*.<br>
It measures how well-defined the clusters are by quantifying how similar each data point is to its own cluster compared to other clusters.
A higher Silhouette Score means that clusters are well-separated and cohesive, while a lower score indicates that clusters are overlapping or poorly defined.



In [ ]:
silhouette_scores = []
inertia = []
k_values = range(2, 20)
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(pca_features)
    inertia.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(pca_features, kmeans.labels_))
    print(f"K={k}, Inertia={kmeans.inertia_}, Silhouette Score={silhouette_scores[-1]}")

knee_locator = KneeLocator(k_values, inertia, curve="convex", direction="decreasing")
optimal_k_elbow = knee_locator.elbow
if optimal_k_elbow:
    elbow_idx = k_values.index(optimal_k_elbow)
    
    second_derivative = np.diff(np.diff(inertia))
    
    if elbow_idx > 0 and elbow_idx < len(second_derivative):
        elbow_strength = second_derivative[elbow_idx - 1]
    else:
        elbow_strength = 0
    
    inertia_drop = (inertia[elbow_idx - 1] - inertia[elbow_idx]) / inertia[elbow_idx - 1]

    if elbow_strength > np.percentile(second_derivative, 75) and inertia_drop > 0.3:
        optimal_k = optimal_k_elbow
        print(f"✅ Optimal K determined by Elbow Method: {optimal_k}")
    else:
        optimal_k = k_values[np.argmax(silhouette_scores)]
        print(f"🔄 Switching to Silhouette Score: Optimal K = {optimal_k}")
else:
    optimal_k = k_values[np.argmax(silhouette_scores)]
    print(f"🔄 No clear elbow detected. Using Silhouette Score: Optimal K = {optimal_k}")

We can plot the graphs of Inertia and Silhouette score

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(k_values, inertia, marker='o', linestyle='--', label="Inertia")
if optimal_k_elbow:
    plt.axvline(x=optimal_k_elbow, color='red', linestyle='--', label="Elbow Point")
plt.xlabel("Number of Clusters (K)")
plt.ylabel("Inertia")
plt.title("Elbow Method for Optimal K")
plt.grid(True)
plt.legend()
plt.show()

plt.figure(figsize=(8, 5))
plt.plot(k_values, silhouette_scores, marker='o', linestyle='--', label="Silhouette Score")
plt.axvline(x=optimal_k, color='green', linestyle='--', label="Chosen K")
plt.xlabel("Number of Clusters (K)")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Method for Optimal K")
plt.grid(True)
plt.legend()
plt.show()

## Finally Clustering
Now that we found our number of clusters we can finally train our model and output the results

In [ ]:
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
kmeans_labels = kmeans.fit_predict(pca_features)

for i, website in enumerate(valid_websites):
    cluster_label = kmeans_labels[i]
    print(f"{website} -> Cluster {cluster_label}")

# Evaluating the model
Taking a close look at the output, it can be noticed that the model performs well at detecting the main clusters of websites, however it is sensitive to some noise points (unique sites are misclasiffied with some big groups). <br>
Also we can plot the first 2 principal components and take a look at the clusters.

In [ ]:
pca_2d = PCA(n_components=2)
reduced_features = pca_2d.fit_transform(pca_features)

plt.figure(figsize=(10, 7))
scatter = plt.scatter(reduced_features[:, 0], reduced_features[:, 1], 
                      c=kmeans_labels, cmap='viridis', s=50, edgecolors='k')

color_bar = plt.colorbar(scatter)
color_bar.set_label('Cluster Label')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('2D K-Means Clustering of Websites')
plt.show()